<h1><center>  股票情绪  -  Nucleus API用例</center></h1>


<h1><center>  所有权及保密条款属SumUp Analytics所有</center></h1>
<h1><center>  免责声明和服务条款可通过 www.sumup.ai 获取</center></h1>


#  

 


## 目标: 
-	使用卖方研究报告衡量特定股票的情绪


## 数据:
-	卖方股票研究报告
-   SEC备案,比如8K



## Nucleus APIs used:
-	数据集创建 API
 - 	*api_instance.post_upload_file(file, dataset)*
 - 	*nucleus_helper.import_files(api_instance, dataset, file_iters, processes=1)*

        nucleus_helper.import_files利用api_instance.post_upload_file并行执行来加速数据集的创建
        

-	文件情绪 API
 - 	*api_instance.post_doc_sentiment_api(payload)*



-	主题建模 API
 - 	*api_instance.post_topic_api(payload)*



-	DocInfo API
 - 	*api_instance.post_doc_info(payload)*



## 方法:

### 1.	数据集准备
-	创建一个Nucleus数据集，其中包含所选历史时期内的所有相关文档

    

In [ ]:
import csv
import json
import nucleus_api.api.nucleus_api as nucleus_helper
import nucleus_api
from nucleus_api.rest import ApiException

configuration = nucleus_api.Configuration()
configuration.host = 'UPDATE-WITH-API-SERVER-HOSTNAME'
configuration.api_key['x-api-key'] = 'UPDATE-WITH-API-KEY'

# 创建API实例
api_instance = nucleus_api.NucleusApi(nucleus_api.ApiClient(configuration))

In [ ]:
print('--------- Append all files from local folder to dataset in parallel -----------')
folder = 'Sellside_research'         
dataset = 'Sellside_research'# str | 文件的目标数据集。

# 以递归方式从文件夹构建文件。 
# 每一项采用以下格式：
# {'filename': filename,   # 要上传的文件名。 需要
#  'metadata': {           # 该文件的元数据。 可选
#      'key1': val1,       # 只要名称，密钥就可以有任意名称
#      'key2': val2        # 包含字母数字（0-9 | a-z | A-Z）和下划线（_）
#   } 
# }
file_iter = []
for root, dirs, files in os.walk(folder):
    for file in files:
        #if Path(file).suffix == '.pdf': # .txt .doc .docx .rtf .html .csv also supported
            file_dict = {'filename': os.path.join(root, file),
                         'metadata': {'ticker': 'AAPL',
                                      'company': 'Apple',
                                      'bank': 'Credit Suisse',
                                      'category': 'sell side research'
                                      'date': '2019-01-01'}}
            file_iter.append(file_dict)

file_props = nucleus_helper.upload_files(api_instance, dataset, file_iter, processes=4)
for fp in file_props:
    print(fp.filename, '(', fp.size, 'bytes) has been added to dataset', dataset)

-	使用SEC文件和嵌入式Nucleus数据传输的替代方案

In [ ]:
dataset = "Corporate_docs" 
period_start = "2010-01-01" 
period_end= "2019-06-01"

payload = nucleus_api.EdgarQuery(destination_dataset=dataset,
                                tickers=["FB", "AMZN", "INTL", "IBM", "NFLX", "GOOG"], 
                                filing_types=["8-K", "8-K/A"], 
                                sections=[],
                                period_start=period_start,
                                period_end=period_end)

api_response = api_instance.post_create_dataset_from_sec_filings(payload)

**您可以随后直接在API中处理数据集中的特定时间段，如下所示**

### 2.	衡量一只股票的情绪
- 选择与库存相关的所有文档


- 衡量每个讨论公司的文件的情绪


- 将报告的情绪汇总到公司的情绪中
 
 
- 接下来，我们将讨论如何利用用户可用的不同参数来优化此分析




In [ ]:
# 提取与所选公司相关的所有文档
import numpy as np

payload = nucleus_api.DocInfo(dataset='Sellside_research',
                             metadata_selection={'ticker': 'AAPL'})
api_response = api_instance.post_doc_info(payload)

doc_list = []
for res in api_response.result:        
    doc_list.append(res.title) 

doc_list = np.unique(doc_list)

print('-------- Get the sentiment of each document ----------------')
reports_sentiment = []
for i in range(len(company_list)):
    payload = nucleus_api.DocumentSentimentModel(dataset='Sellside_research', 
                                                doc_title=doc_list[i], 
                                                custom_stop_words="", 
                                                num_topics=10, 
                                                num_keywords=10)
    try:
        api_response = api_instance.post_doc_sentiment_api(payload)
        api_ok = True
    except ApiException as e:
        api_error = json.loads(e.body)
        print('ERROR:', api_error['message'])
        api_ok = False

    if api_ok:    
        reports_sentiment.append(api_response.result.sentiment)

# 将每份报告中的情绪加入公司的情绪中
company_sentiment = np.mean(reports_sentiment, axis=1)

-	Repeat the above tasks for each company you are interested in

In [ ]:
import numpy as np

# 您感兴趣的公司名单
company_list = ['AAPL', 'GOOG', 'FB', 'BABA', 'NFLX']

# 仔细检查每一个并获得情绪
company_sentiment = []
for i in range(len(company_list)):  
    # 获取讨论给定公司的所有文档
    payload = nucleus_api.DocInfo(dataset='Sellside_research',
                                 metadata_selection={'ticker': company_list[i]})
    try:
        api_response = api_instance.post_doc_info(payload)
        api0_ok = True
    except ApiException as e:
        api_error = json.loads(e.body)
        print('ERROR:', api_error['message'])
        api0_ok = False

    if api0_ok:
        doc_list = []
        for res in api_response.result:        
            doc_list.append(res.title) 

        doc_list = np.unique(doc_list)

        # 获取每个文档的情绪
        reports_sentiment = []
        for j in range(len(company_list)):
            payload = nucleus_api.DocumentSentimentModel(dataset='Sellside_research', 
                                                        doc_title=doc_list[j], 
                                                        custom_stop_words="", 
                                                        num_topics=10, 
                                                        num_keywords=10)
            try:
                api_response = api_instance.post_doc_sentiment_api(payload)
                api_ok = True
            except ApiException as e:
                api_error = json.loads(e.body)
                print('ERROR:', api_error['message'])
                api_ok = False

            if api_ok:
                reports_sentiment.append(api_response.result.sentiment)

    # 将每份报告中的情绪加入公司的情绪中
    company_sentiment.append(np.mean(reports_sentiment, axis=1))

### 3.	结果解释
-	在一个行业部门中绘制公司情绪的时间序列; 或针对每家公司的股票回报序列

### 4.	微调

#### a.	量身定制主题
-   通过使用Contrast Analysis API中的`custom_stop_words`参数，排除无关紧要或次要主题，以定制您公司的情绪


-	在文档子集中提取对比主题的关键字并打印这些主题的关键字



In [ ]:
print('------------- Get list of topics from dataset --------------')

payload = nucleus_api.Topics(dataset='Sellside_research',                       
                            query='',                       
                            num_topics=20, 
                            num_keywords=5,
                            metadata_selection={'ticker': 'AAPL'})
try:
    api_response = api_instance.post_topic_api(payload)        
    api_ok = True
except ApiException as e:
    api_error = json.loads(e.body)
    print('ERROR:', api_error['message'])
    api_ok = False

if api_ok:    
    for i, res in enumerate(api_response.result.topics):
        print('Topic', i, ' keywords: ', res.keywords)    
        print('---------------')

然后，您可以通过创建custom_stop_words变量来定制公司的情绪。 如下所示，将初始化变量传递到第2节主代码的有效负载中：

In [1]:
custom_stop_words = ["call","report"] # str | 停用词列表。 （可选）

#### b.	探索文档类型，回顾期，提取的主题数量的影响

**num_topics**: 您可以通过更改第2部分主要代码中有效负载中的变量`num_topics`来使用不同广度的主题来计算公司的情绪。
- 在建立情绪时，较大的值将包含更广泛的主题/内容，而较小的值将提供具有主要主题的浓缩度量。
- 如果`num_topics`太大，一些非常边缘的话题可能会在衡量公司情绪方面带来很多噪音。


**Document types**: 您可以使用卖方研究，新闻或公司出版物来调查公司的情绪如何变化。 在那些不同的数据集上重新运行第2节的主要代码。 您还可以构建包含跨提供程序的所有内容的数据集，然后使用`metadata_selection`仅选择某些类型的文档：

In [ ]:
metadata_selection = {"category": "News"}   # str | json object of {\"metadata_field\":[\"selected_values\"]} (optional)

### 5.	下一步
-	可能的扩展：针对不同的行业部门重复上述任务
 - 汇总某个行业中每家公司的情绪，以获得行业情绪

版权(c) 2019年 SumUp Analytics 公司 版权所有。

通知：所有信息均属于SumUp Analytics Inc公司及其供应商的财产。 本合同所包含的知识产权和技术概念属于SumUp Analytics Inc.及其供应商的专利，可由美国和外国专利、在工艺中的专利以及受贸易秘密或版权法保护的专利涵盖。

除非得到SumUp Analytics公司的事先书面批准，否则严禁传播此类信息或复制此材料。